In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np, pandas as pd
from fbprophet import Prophet
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from pmdarima.arima.utils import ndiffs
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

C:\Users\kseni\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
C:\Users\kseni\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


In [2]:
#DATA PREPARATION FOR UNIVARIATE PROPHET MODEL

#GATHER Paris DATA FROM ALL AVAILABLE YEARS
Paris_Data = pd.read_csv('Paris_data.csv')
Paris_Data.rename(columns = {Paris_Data.columns[0]:'ds'}, inplace = True)

#KEEP ONLY THE POLLUTANTS COLUMNS
cols=['pm25', 'pm10', 'no2', 'o3', 'so2', 'ds']

#MAKE VALUES NUMERIC
Paris_Data_uni = Paris_Data.loc[:, Paris_Data.columns.intersection(cols)]
Paris_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']] = Paris_Data_uni[['pm25', 'pm10', 'no2', 'o3', 'so2']].apply(pd.to_numeric, errors='coerce')

#FILL NAN VALUES
Paris_Data_uni = Paris_Data_uni.iloc[1: , :]

#SOME VALUES ARE EMPTY, BUT NOT RECOGNIZED AS NAN. REPLACE EMPTY VALUES WITH NAN TO FILL THEM LATER
Paris_Data_uni = Paris_Data_uni.replace(r'^\s*$', np.nan, regex=True)
Paris_Data_uni = Paris_Data_uni.fillna(method='ffill')
Paris_Data_uni = Paris_Data_uni.fillna(method='bfill')

#REMOVE DUPLICATE VALUES
Paris_Data_uni = Paris_Data_uni[~Paris_Data_uni.index.duplicated(keep='first')]

#Fb prophet model predicts univariate time series. It requires that the Date index column is named 'ds'
#and the values column is named 'y'
Paris_Data_uni['ds'] = pd.DatetimeIndex(Paris_Data_uni['ds'])

In [3]:
Paris_Data_uni

,ds,no2,o3,pm10,pm25,so2
1,2014-12-30,0.567033,0.086777,0.338235,0.372549,0.204082
2,2014-12-31,0.778022,0.070248,0.544118,0.542484,0.204082
3,2015-01-01,0.679121,0.188017,0.529412,0.666667,0.153061
4,2015-01-02,0.567033,0.128099,0.308824,0.405229,0.102041
5,2015-01-03,0.356044,0.163223,0.102941,0.228758,0.051020
...,...,...,...,...,...,...
1869,2022-03-14,0.285714,0.450413,0.161765,0.156863,0.020408
1870,2022-03-15,0.463736,0.198347,0.147059,0.235294,0.020408
1871,2022-03-16,0.373626,0.126033,0.397059,0.326797,0.020408
1872,2022-03-17,0.178022,0.469008,0.147059,0.183007,0.020408


In [4]:
#Check data for stationarity to apply the ARIMA model.

#The null hypothesis is that the time series is non-stationary.
#Using the Augmented Dickey Fuller test (adfuller()) we can check for stationarity.
#If p-value < significance level(0.05), the we reject the null hypothesis.
#Otherwise, the time series is non-stationary and needs differencing.

print("Check if Paris_Data is stationary :")
for col in ['no2', 'o3', 'so2', 'pm10', 'pm25']:
    result = adfuller(Paris_Data[col])
    print('p-value for ' + col + ' : %f' % result[1])

Check if Paris_Data is stationary :
p-value for no2 : 0.000088
p-value for o3 : 0.000176
p-value for so2 : 0.000694
p-value for pm10 : 0.000000
p-value for pm25 : 0.000000


In [5]:
def fbprophet_predict(time_series, col):
    
    #CREATE MODEL
    model = Prophet()
    
    #SPLIT TO TRAIN AND TEST PORTIONS
    train_size = 0.8
    train = time_series[:int(train_size*(len(time_series)))]
    test = time_series[int(train_size*(len(time_series))):]
    
    #FIT TRAIN DATA TO MODEL
    model.fit(train)
    
    #PREDICT THE TEST PORTION. THE PARAMETER IN THE PREDICT FUNCTION IS THE TIME INTERVAL TO PREDICT
    forecast = model.predict(test[['ds']])
    
    #FORECAST PREVIEW. 'ds' IS THE DAILY INDEX, 'yhat' IS THE FORECAST, 'yhat_lower' and 'yhat_upper' ARE THE
    #LOWER AND UPPER BOUND OF THE FORECASTED VALUE RESPECTIVELY.
    print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
    print(test.reset_index(drop=True).head())
    
    #COMPUTE RMSE OF THE FORECASTED AND ACTUAL VALUE
    rmse = math.sqrt(mean_squared_error(test[['y']], forecast[['yhat']]))
    mse = mean_squared_error(test[['y']], forecast[['yhat']])
    mae = mean_absolute_error(test[['y']], forecast[['yhat']])
    
    print("FB Prophet RMSE for Paris[" + col + "]: " + str(rmse) + "\n")
    print("FB Prophet MSE for Paris[" + col + "]: " + str(mse) + "\n")
    print("FB Prophet MAE for Paris[" + col + "]: " + str(mae) + "\n")

    return test, forecast, rmse

In [6]:
#FB PROPHET REQUIRES THE DATAFRAME TO HAVE TWO COLUMNS. 'ds' AND 'y', 'ds' IS THE TIME INDEX AND 'y' IS THE
#VALUE OF THE POLLUTANT

#RENAME COLUMNS TO FIT TO THE PROPHET MODEL
Paris_o3 = Paris_Data_uni[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_Data_uni[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_Data_uni[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_Data_uni[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_Data_uni[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

In [7]:
pollutants = ['o3', 'no2', 'so2', 'pm10', 'pm25']

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.304903    0.133027    0.473561
1 2021-02-12  0.302925    0.139914    0.454507
2 2021-02-13  0.324316    0.164439    0.495614
3 2021-02-14  0.349136    0.184270    0.515212
4 2021-02-15  0.328419    0.171084    0.489618
          ds         y
0 2021-02-11  0.334711
1 2021-02-12  0.390496
2 2021-02-13  0.413223
3 2021-02-14  0.448347
4 2021-02-15  0.210744
FB Prophet RMSE for Paris[o3]: 0.14040385530622787

FB Prophet MSE for Paris[o3]: 0.019713242584852173

FB Prophet MAE for Paris[o3]: 0.11053011715284723



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.281792    0.091726    0.449966
1 2021-02-12  0.297582    0.104908    0.460006
2 2021-02-13  0.225971    0.042866    0.410292
3 2021-02-14  0.165142    0.004135    0.334869
4 2021-02-15  0.231551    0.054795    0.404197
          ds         y
0 2021-02-11  0.283516
1 2021-02-12  0.215385
2 2021-02-13  0.261538
3 2021-02-14  0.226374
4 2021-02-15  0.367033
FB Prophet RMSE for Paris[no2]: 0.15661167644307863

FB Prophet MSE for Paris[no2]: 0.024527217198311548

FB Prophet MAE for Paris[no2]: 0.11484689816159183



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.085145    0.032388    0.141214
1 2021-02-12  0.082198    0.028460    0.138045
2 2021-02-13  0.076322    0.021840    0.138163
3 2021-02-14  0.072999    0.017239    0.131111
4 2021-02-15  0.080989    0.029026    0.137652
          ds         y
0 2021-02-11  0.051020
1 2021-02-12  0.153061
2 2021-02-13  0.071429
3 2021-02-14  0.173469
4 2021-02-15  0.091837
FB Prophet RMSE for Paris[so2]: 0.06615917148026548

FB Prophet MSE for Paris[so2]: 0.004377035970955174

FB Prophet MAE for Paris[so2]: 0.05587873154289234



INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.253615    0.075499    0.445055
1 2021-02-12  0.270857    0.097680    0.442510
2 2021-02-13  0.227219    0.049527    0.406144
3 2021-02-14  0.196381    0.023545    0.362557
4 2021-02-15  0.222861    0.047969    0.416506
          ds         y
0 2021-02-11  0.264706
1 2021-02-12  0.205882
2 2021-02-13  0.205882
3 2021-02-14  0.220588
4 2021-02-15  0.294118
FB Prophet RMSE for Paris[pm10]: 0.13933367835111757

FB Prophet MSE for Paris[pm10]: 0.019413873922852688

FB Prophet MAE for Paris[pm10]: 0.10222987534025897

          ds      yhat  yhat_lower  yhat_upper
0 2021-02-11  0.281457    0.100330    0.472099
1 2021-02-12  0.293338    0.111939    0.478741
2 2021-02-13  0.282900    0.095155    0.468369
3 2021-02-14  0.259985    0.081224    0.439707
4 2021-02-15  0.254184    0.061967    0.445062
          ds         y
0 2021-02-11  0.359477
1 2021-02-12  0.274510
2 2021-02-13  0.274510
3 2021-02-14  0.313725
4 2021-02-15  0.313725

### Repeat process but this time train model with the same amount of data as Athens, to prove that the model returns better results due to bigger amount of data.

In [8]:
import pandas as pd
Athens_Data = pd.read_csv('Athens_Data.csv', index_col=0)

In [9]:
len(Athens_Data)

839

In [10]:
data_to_drop = len(Paris_Data) - len(Athens_Data)

In [11]:
Paris_less_data = Paris_Data.iloc[data_to_drop: , :]
len(Paris_less_data)

839

In [12]:
Paris_less_data

,ds,no2,o3,pm10,pm25,so2,temp,dew,humidity,windspeed,winddir
1035,2019-11-05,0.397802,0.291322,0.176471,0.176471,0.091837,0.360000,0.614035,0.770794,0.251908,0.604913
1036,2019-11-06,0.349451,0.171488,0.176471,0.254902,0.040816,0.376000,0.640351,0.799524,0.170483,0.733057
1037,2019-11-07,0.259341,0.295455,0.102941,0.124183,0.030612,0.344000,0.578947,0.726984,0.455471,0.595146
1038,2019-11-08,0.479121,0.107438,0.205882,0.228758,0.010204,0.277333,0.532164,0.786825,0.129771,0.413732
1039,2019-11-09,0.364835,0.165289,0.235294,0.300654,0.020408,0.266667,0.529240,0.817778,0.302799,0.632436
...,...,...,...,...,...,...,...,...,...,...,...
1869,2022-03-14,0.285714,0.450413,0.161765,0.156863,0.020408,0.357333,0.540936,0.603175,0.206107,0.444510
1870,2022-03-15,0.463736,0.198347,0.147059,0.235294,0.020408,0.373333,0.652047,0.847619,0.170483,0.311927
1871,2022-03-16,0.373626,0.126033,0.397059,0.326797,0.020408,0.402667,0.666667,0.793651,0.376590,0.438591
1872,2022-03-17,0.178022,0.469008,0.147059,0.183007,0.020408,0.338667,0.540936,0.614286,0.402036,0.371412


In [13]:
Paris_o3 = Paris_less_data[['ds', 'o3']]
Paris_o3.rename(columns = {'o3':'y'}, inplace = True)

Paris_no2 = Paris_less_data[['ds', 'no2']]
Paris_no2.rename(columns = {'no2':'y'}, inplace = True)

Paris_so2 = Paris_less_data[['ds', 'so2']]
Paris_so2.rename(columns = {'so2':'y'}, inplace = True)

Paris_pm10 = Paris_less_data[['ds', 'pm10']]
Paris_pm10.rename(columns = {'pm10':'y'}, inplace = True)

Paris_pm25 = Paris_less_data[['ds', 'pm25']]
Paris_pm25.rename(columns = {'pm25':'y'}, inplace = True)

datasets = [Paris_o3, Paris_no2, Paris_so2, Paris_pm10, Paris_pm25]

i=0

#EVALUATE MODEL FOR EVERY POLLUTANT
for data in datasets:
    test, forecast, error = fbprophet_predict(data, pollutants[i])
    i = i+1

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.566563    0.395057    0.730692
1 2021-09-07  0.568175    0.375112    0.769257
2 2021-09-08  0.585721    0.397333    0.776807
3 2021-09-09  0.583086    0.399606    0.769654
4 2021-09-10  0.583267    0.396690    0.777080
           ds         y
0  2021-09-06  0.617769
1  2021-09-07  0.595041
2  2021-09-08  0.719008
3  2021-09-09  0.489669
4  2021-09-10  0.433884
FB Prophet RMSE for Paris[o3]: 0.4361973760740367

FB Prophet MSE for Paris[o3]: 0.19026815089387458

FB Prophet MAE for Paris[o3]: 0.39683500854401044



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.143381   -0.021107    0.301529
1 2021-09-07  0.180431    0.029733    0.329727
2 2021-09-08  0.175753    0.021417    0.331948
3 2021-09-09  0.176274    0.016202    0.340490
4 2021-09-10  0.165014    0.005157    0.317945
           ds         y
0  2021-09-06  0.470330
1  2021-09-07  0.404396
2  2021-09-08  0.283516
3  2021-09-09  0.232967
4  2021-09-10  0.186813
FB Prophet RMSE for Paris[no2]: 0.24653623502169442

FB Prophet MSE for Paris[no2]: 0.06078011517867215

FB Prophet MAE for Paris[no2]: 0.21234174186508983



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.040428   -0.024631    0.103033
1 2021-09-07  0.040146   -0.028610    0.102495
2 2021-09-08  0.033680   -0.031820    0.097394
3 2021-09-09  0.046439   -0.019750    0.111304
4 2021-09-10  0.037743   -0.031648    0.105225
           ds         y
0  2021-09-06  0.061224
1  2021-09-07  0.020408
2  2021-09-08  0.020408
3  2021-09-09  0.020408
4  2021-09-10  0.020408
FB Prophet RMSE for Paris[so2]: 0.057367997211116524

FB Prophet MSE for Paris[so2]: 0.0032910871040146734

FB Prophet MAE for Paris[so2]: 0.02509709309654034



INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.187531    0.028038    0.328446
1 2021-09-07  0.204419    0.038914    0.371803
2 2021-09-08  0.221535    0.068309    0.374267
3 2021-09-09  0.203323    0.049094    0.367365
4 2021-09-10  0.199551    0.055208    0.354993
           ds         y
0  2021-09-06  0.279412
1  2021-09-07  0.294118
2  2021-09-08  0.308824
3  2021-09-09  0.220588
4  2021-09-10  0.117647
FB Prophet RMSE for Paris[pm10]: 0.14458849124806047

FB Prophet MSE for Paris[pm10]: 0.02090583180139046

FB Prophet MAE for Paris[pm10]: 0.10604562368566056

          ds      yhat  yhat_lower  yhat_upper
0 2021-09-06  0.166257    0.011221    0.309017
1 2021-09-07  0.180823    0.028495    0.335931
2 2021-09-08  0.194913    0.033444    0.343473
3 2021-09-09  0.175504    0.036794    0.327697
4 2021-09-10  0.177639    0.026568    0.326869
           ds         y
0  2021-09-06  0.281046
1  2021-09-07  0.281046
2  2021-09-08  0.313725
3  2021-09-09  0.281046
4  2021-09-1